In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from ipywidgets import SelectMultiple, interact

In [25]:
df = pd.read_csv('../decklizer/tests/unit_test_positive_loss.csv', index_col=0)
df = df.reset_index(drop=True)
df.head()

,B,L,n,q,w,loss,jumbos,inventory,summary,max_combo,layouts,target,edge
0,1522,46167,"[13, 13, 19]","[35, 76, 11]","[839, 744, 1392]",20.827076,84,"{826: 0.0, 731: 0.0, 1373: 0.0}",loss jumbos layout\n...,2,3,1,0
1,1522,46167,"[13, 13, 19]","[35, 76, 11]","[839, 744, 1392]",20.827076,84,"{826: 0.0, 731: 0.0, 1373: 0.0}",loss jumbos layout\n...,3,3,1,0
2,7902,71536,"[18, 12, 20, 4, 16]","[73, 83, 35, 26, 38]","[7390, 7417, 2125, 1071, 2666]",8.041597,210,"{1415: 0.0, 2952: 0.0, 728: 3.0, 3302: 0.0, 13...",loss jumbos ...,2,6,1,0
3,7902,71536,"[18, 12, 20, 4, 16]","[73, 83, 35, 26, 38]","[7390, 7417, 2125, 1071, 2666]",8.041597,210,"{1415: 0.0, 2952: 0.0, 728: 3.0, 3302: 0.0, 13...",loss jumbos ...,3,6,1,0
4,7902,71536,"[18, 12, 20, 4, 16]","[73, 83, 35, 26, 38]","[7390, 7417, 2125, 1071, 2666]",8.079896,188,"{7372: 0.0, 7405: 0.0, 2105: 1.0, 1067: 2.0, 2...",loss jumbos ...,2,5,1,0


In [26]:
df.loc[df['loss'] < 0].shape

(0, 13)

In [56]:
df['all slits less than 1/3 bin width'] = False
for row in df.index:
    df.iloc[row,-1] = all([df.iloc[row]['B']/3 > int(x) for x in eval(df.iloc[row]['w'])])
df['all slits less than 1/4 bin width'] = False
for row in df.index:
    df.iloc[row,-1] = all([df.iloc[row]['B']/4 > int(x) for x in eval(df.iloc[row]['w'])])
df['all slits less than 1/2 bin width'] = False
for row in df.index:
    df.iloc[row,-1] = all([df.iloc[row]['B']/2 > int(x) for x in eval(df.iloc[row]['w'])])
df['all slits less than 2/3 bin width'] = False
for row in df.index:
    df.iloc[row,-1] = all([df.iloc[row]['B']/3*2 > int(x) for x in eval(df.iloc[row]['w'])])
df['all slits less than 3/4 bin width'] = False
for row in df.index:
    df.iloc[row,-1] = all([df.iloc[row]['B']/4*3 > int(x) for x in eval(df.iloc[row]['w'])])
df['all slits less than 1/5 bin width'] = False
for row in df.index:
    df.iloc[row,-1] = all([df.iloc[row]['B']/5 > int(x) for x in eval(df.iloc[row]['w'])])

In [ ]:
df[df.columns]

In [51]:
layouts_selector = SelectMultiple(
    options=list(np.sort(df['layouts'].unique().astype(int))),
    value=list(np.sort(df['layouts'].unique().astype(int)))
)

In [54]:
def hist_plot(layouts=layouts_selector):
    fig, ax = plt.subplots(1, 1, figsize=(10, 10))
    
    df.loc[(final['layouts'].isin(layouts)) & (df['all slits less than .5 bin width'] == True)]\
        .plot(kind='hist', y='loss', bins=20, ax=ax)

In [55]:
interact(hist_plot)

interactive(children=(SelectMultiple(description='layouts', index=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10), options=…

<function __main__.hist_plot(layouts=SelectMultiple(description='layouts', index=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10), options=(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11), value=(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11)))>

In [ ]:
df

In [ ]:
goal = 3
verbiose = True
bucket = df.iloc[189][['w', 'n', 'L', 'q', 'B', 'max_combo', 
                                         'layouts', 'target', 'edge']].reset_index(drop=True).iloc[0]
patterns, layout = seed_patterns(eval(bucket['w']), 
                                     [math.ceil(i*bucket['target']) 
                                      for i in eval(bucket['q'])], 
                                     int(bucket['B'])+(int(bucket['edge'])*2), 
                                     eval(bucket['n']), 
                                     max_combinations=int(bucket['max_combo']), 
                                     goal=goal, 
                                     verbiose=verbiose)

In [ ]:
loss, inv, summary = find_optimum(patterns, 
                                      layout, 
                                      eval(bucket['w']), 
                                      [math.ceil(i*bucket['target']) 
                                      for i in eval(bucket['q'])], 
                                      int(bucket['B'])+(int(bucket['edge'])*2), 
                                     eval(bucket['n']),
                                      bucket['L'],
                                    max_combinations=int(bucket['max_combo']), 
                                    max_patterns=3,
                                    prioritize='time')